In [17]:
import cv2
import numpy as np
import os

def is_high_frequency(image_path, threshold):
    # Read the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply the Laplacian operator to detect edges (high frequency)
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    
    # Compute the variance of the Laplacian
    variance = laplacian.var()
    print(variance)
    
    # Check if the variance exceeds the threshold for high frequency
    return variance > threshold

def test_images_for_high_frequency(directory):
    results = {}
    
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):  # Add more image extensions if needed
            image_path = os.path.join(directory, filename)
            results[filename] = is_high_frequency(image_path, 2600)
    
    return results

# Example usage
directory = "../drive-download-20240804T174950Z-001"
threshold_value = 100  # You can adjust this value based on your needs
results = test_images_for_high_frequency(directory)

for image_name, is_high_freq in results.items():
    print(f"{image_name}: {'High Frequency' if is_high_freq else 'Not High Frequency'}")


152.7555657010068
87.06702032763033
1684.1032745932814
2365.0925071750316
313.93799722222224
471.04825122059145
1238.2320810062108
504.81498110176665
62.51663881888483
94912.23380675654
625.7643830997223
2714.9994946116685
113.64366729987529
6.629233319722221
2158.939750475178
690.5450835275147
655.2374565108332
5825.503730571903
112641.84035053477
01 - Straightforward.jpg: Not High Frequency
02 - You can do it.jpg: Not High Frequency
03 - Should be okay.jpg: Not High Frequency
04 - Still ok, I hope.jpg: Not High Frequency
05 - Looks cool, hope it runs cool too.jpg: Not High Frequency
06 - Hatetlewe7 hatlewe7.jpg: Not High Frequency
07 - Hatet3eweg hat3eweg.jpg: Not High Frequency
08 - Ew3a soba3ak ya3am.jpg: Not High Frequency
09 - El spero spathis we23et 3aaaa.jpg: Not High Frequency
10 - Mal7 w Felfel.jpg: High Frequency
11 - Ya setty ew3i.jpg: Not High Frequency
12 - weewooweewoo.jpg: High Frequency
13 - Matozbot el camera ya Kimo.jpg: Not High Frequency
14 - 2el noor 2ata3.jpg: No